# Классификация текстов: спам-фильтр для SMS

В этом задании вам предстоит взять открытый датасет с SMS-сообщениями, размеченными на спам ("spam") и не спам ("ham"), построить на нем классификатор текстов на эти два класса, оценить его качество с помощью кросс-валидации, протестировать его работу на отдельных примерах, и посмотреть, что будет происходить с качеством, если менять параметры вашей модели.

## Задание

1. Загрузите датасет. Описание датасета можно посмотреть [здесь](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection).

2. Считайте датасет в Python (можете сразу грузить все в память, выборка небольшая), выясните, что используется в качестве разделителей и как проставляются метки классов.

3. Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

4. Используя `sklearn.feature_extraction.text.CountVectorizer` со стандартными настройками, получите из списка текстов матрицу признаков `X`.

5. Оцените качество классификации текстов с помощью `LogisticRegression()` с параметрами по умолчанию, используя `sklearn.cross_validation.cross_val_score` и посчитав среднее арифметическое качества на отдельных fold'ах. Установите `random_state=2`. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

6. А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

```
"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"
```

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

7. Задайте в `CountVectorizer` параметр `ngram_range=(2,2)`, затем `ngram_range=(3,3)`, затем `ngram_range=(1,3)`. Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

8. Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии `MultinomialNB()`. Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

По какой-то причине обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно посчитайте частоты слов и обучите классификатор.

9. Попробуйте использовать в логистической регрессии в качестве признаков `Tf*idf` из `TfidfVectorizer` на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с `CountVectorizer` на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к `tf*idf` не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [52]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB

In [20]:
data = pd.read_csv('SMSSpamCollection.txt.gz', compression='gzip', delimiter='\t', header=None)
data.columns=['target', 'text']
data.head()

,target,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [21]:
le = LabelEncoder()
Y = le.fit_transform(data.target)

In [41]:
pipeline1 = Pipeline(
            [("vectorizer", CountVectorizer()),
            ("classifier", LogisticRegression(random_state=2))]
        )

score = cross_val_score(pipeline1, data.text, Y, cv=10, scoring='f1')
answer1 = score.mean()

print(answer1)
with open('answer1.txt', 'w') as fout:
    fout.write(str(np.round(answer1, 1)))

0.932640298361


In [32]:
pipeline1.fit(data.text, Y)

Pipeline(steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...nalty='l2', random_state=2, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [40]:
test = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
"FreeMsg: Txt: claim your reward of 3 hours talk time",
"Have you visited the last lecture on physics?",
"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
"Only 99$"]

pred = pipeline1.predict(test)
print(pred)
p = " ".join([str(p) for p in pred])

with open('answer2.txt', 'w') as fout:
    fout.write(p)

[1 1 0 0 0]


In [45]:
scores = []
for ngram_range in [(2,2), (3,3), (1,3)]:
    pipeline = Pipeline([
            ("vectorizer", CountVectorizer(ngram_range=ngram_range)),
            ("classifier", LogisticRegression(random_state=2))
        ])

    score = cross_val_score(pipeline, data.text, Y, cv=10, scoring='f1')
    print("{} {}".format(ngram_range, score.mean()))
    scores.append(np.round(score.mean(), 2))

with open('answer3.txt', 'w') as fout:
    fout.write(" ".join([str(s) for s in scores]))

(2, 2) 0.8224220664187133
(3, 3) 0.7250161555467377
(1, 3) 0.9251382558648837


In [50]:
scores = []
for ngram_range in [(2,2), (3,3), (1,3)]:
    vec = CountVectorizer(ngram_range=ngram_range)
    X = vec.fit_transform(data.text)
    clf = MultinomialNB()
    score = cross_val_score(clf, X, Y, cv=10, scoring='f1')
    print("{} {}".format(ngram_range, score.mean()))
    scores.append(np.round(score.mean(), 2))
    
with open('answer4.txt', 'w') as fout:
    fout.write(" ".join([str(s) for s in scores]))

(2, 2) 0.6455015177985443
(3, 3) 0.37871948524573595
(1, 3) 0.8884859656061002


In [57]:
pipeline = Pipeline([
        ("vectorizer", TfidfVectorizer()),
        ("classifier", LogisticRegression(random_state=2))
    ])

score = cross_val_score(pipeline, data.text, Y, cv=10, scoring='f1')
print(score.mean())

with open('answer5.txt', 'w') as fout:
    fout.write('-1')

0.878510045534
